In [1]:
import kagglehub
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.ensemble import IsolationForest

c:\Users\51206614\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
path = kagglehub.dataset_download("bejopamungkas/transaksi-pembelian-penjualan-sembako")
df = pd.read_csv(path + '\\penjualan barang.csv')

In [3]:
features = df[['kuantum', 'nominal']]
model = IsolationForest(n_estimators=100, contamination=0.05, random_state=50)
model.fit(features)

df['outlier'] = model.predict(features)
outliers = df[df['outlier'] == -1]

In [5]:
def train_model(df, outlier):
    features = df[['kuantum','nominal']] 
    model = IsolationForest(n_estimators=100, contamination=outlier, random_state=50)
    model.fit(features) 
    return model

def detect_fraud(model, df):
    df['fraud'] = model.predict(df[['kuantum','nominal']]) 
    df['fraud'] = df['fraud'].apply(lambda x: "Fraud" if x == -1 else "Normal")  # -1 artinya fraud
    return df

model = train_model(df,(outliers.shape[0]/df.shape[0]))
df_with_fraud = detect_fraud(model, df)
total_fraud = df_with_fraud.loc[df_with_fraud['fraud'] == 'Fraud'].shape[0]


print('Hasil Deteksi Penipuan:')
print(f'Tedapat {total_fraud} data fraud.')

Hasil Deteksi Penipuan:
Tedapat 65 data fraud.


In [6]:
fraud = df_with_fraud.loc[df_with_fraud['fraud'] == 'Fraud']

fraud_summary = fraud.groupby('nama.pembeli').agg(
    jumlah_transaksi=('nama.barang', 'count'),
    total_kuantum=('kuantum', 'sum'),
    total_nominal=('nominal', 'sum')
).reset_index()


In [23]:
fraud_summary.sort_values('jumlah_transaksi', ascending=False)

,nama.pembeli,jumlah_transaksi,total_kuantum,total_nominal
1,TOKO AIDI K.,8,184060.0,1.931914e+09
13,TOKO ENDANG SUSILAWATI,6,78000.0,8.281100e+08
19,TOKO JESAYA GINTING,3,35000.0,3.675000e+08
4,TOKO ANTONIUS,3,35000.0,3.675000e+08
3,TOKO ANDRI SUGIANTO,3,34500.0,3.622500e+08
9,TOKO BERKAT USAHA,3,25500.0,3.060000e+08
15,TOKO FAKHLI HARUDI,3,33000.0,3.462450e+08
5,TOKO APO,3,26500.0,3.180000e+08
17,TOKO HERWIN,3,44560.0,4.678800e+08
30,TOKO SUYONO (BINTANG KAPUAS),2,21000.0,2.520000e+08
